In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_mal
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/'

    train_data = read_task(dataset_location , split = 'mal_train_negative_augmented')
    dev_data = read_task(dataset_location , split = 'mal_sentiment_dev')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_mal
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['ai4bharat/indic-bert']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Malayalam' + 'Augmented'+str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'predictions.bias', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

Training epoch: 1
FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 139 of 632
FINSIHED BATCH: 159 of 632
FINSIHED BATCH: 179 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 1.28378164768219
DEV ACC: 0.5319940476190477
DEV Precision: 0.442030839735173
DEV Recall: 0.451985438563384


Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'predictions.bias', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

Training epoch: 1
FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 139 of 632
FINSIHED BATCH: 159 of 632
FINSIHED BATCH: 179 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 1.1461269855499268
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BA

FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 1.371802806854248
DEV ACC: 0.6106770833333333
DEV Precision: 0.5500791448879269
DEV Recall: 0.5168016578717735
DEV F1Score: 0.5066764210022235
BEST ACCURACY -->  DEV: 0.65011
BEST PRECISION -->  DEV: 0.56631
BEST RECALL -->  DEV: 0.56858
BEST F1SCORE -->  DEV: 0.54263
TIME PER EPOCH: 13.33674220641454

Training epoch: 9
FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 5

...

Training epoch: 1
FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 139 of 632
FINSIHED BATCH: 159 of 632
FINSIHED BATCH: 179 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 1.0100934505462646
DEV ACC: 0.5524553571428571
DEV Precision: 0.4478261080686722
DEV Recall: 0.4586726502955

FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 139 of 632
FINSIHED BATCH: 159 of 632
FINSIHED BATCH: 179 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 1.3973376750946045
DEV ACC: 0.6393229166666667
DEV Precision: 0.5987977947512583
DEV Recall: 0.557032415238823
DEV F1Score: 0.

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'predictions.bias', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

Training epoch: 1
FINSIHED BATCH: 19 of 632
FINSIHED BATCH: 39 of 632
FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 139 of 632
FINSIHED BATCH: 159 of 632
FINSIHED BATCH: 179 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 1.3017553091049194
DEV ACC: 0.4720982142857143
DEV Precision: 0.4374434391803858
DEV Recall: 0.3738440516795

FINSIHED BATCH: 59 of 632
FINSIHED BATCH: 79 of 632
FINSIHED BATCH: 99 of 632
FINSIHED BATCH: 119 of 632
FINSIHED BATCH: 139 of 632
FINSIHED BATCH: 159 of 632
FINSIHED BATCH: 179 of 632
FINSIHED BATCH: 199 of 632
FINSIHED BATCH: 219 of 632
FINSIHED BATCH: 239 of 632
FINSIHED BATCH: 259 of 632
FINSIHED BATCH: 279 of 632
FINSIHED BATCH: 299 of 632
FINSIHED BATCH: 319 of 632
FINSIHED BATCH: 339 of 632
FINSIHED BATCH: 359 of 632
FINSIHED BATCH: 379 of 632
FINSIHED BATCH: 399 of 632
FINSIHED BATCH: 419 of 632
FINSIHED BATCH: 439 of 632
FINSIHED BATCH: 459 of 632
FINSIHED BATCH: 479 of 632
FINSIHED BATCH: 499 of 632
FINSIHED BATCH: 519 of 632
FINSIHED BATCH: 539 of 632
FINSIHED BATCH: 559 of 632
FINSIHED BATCH: 579 of 632
FINSIHED BATCH: 599 of 632
FINSIHED BATCH: 619 of 632
Validation loss per 100 evaluation steps: 1.6801186800003052
DEV ACC: 0.654389880952381
DEV Precision: 0.5782985327151348
DEV Recall: 0.5648700101858618
DEV F1Score: 0.5500869897500145
BEST ACCURACY -->  DEV: 0.67578
BES

In [6]:
print(best_dev_acc)

0.67578125


In [7]:
print(best_epoch)

4


In [8]:
print(best_dev_precision)

0.6018983138855774


In [9]:
print(best_dev_recall)

0.5820071392484735


In [10]:
print(best_dev_f1score)

0.5735926593972456


In [11]:
print(overall_list_dev_acc)

[[0.5319940476190477, 0.6222098214285714, 0.6421130952380952, 0.6443452380952381, 0.6437872023809524, 0.6382068452380952, 0.6197916666666667, 0.6357886904761905, 0.6458333333333333, 0.6244419642857143], [0.5014880952380952, 0.6077008928571429, 0.6285342261904762, 0.5835193452380952, 0.6491815476190476, 0.6501116071428571, 0.611421130952381, 0.6106770833333333, 0.630766369047619, 0.6322544642857143], [0.43880208333333337, 0.6095610119047619, 0.6125372023809524, 0.6486235119047619, 0.6326264880952381, 0.6400669642857143, 0.6415550595238095, 0.6164434523809524, 0.638764880952381, 0.6413690476190476], [0.5524553571428571, 0.615327380952381, 0.6471354166666667, 0.6450892857142857, 0.6437872023809524, 0.6501116071428571, 0.6577380952380952, 0.6393229166666667, 0.6408110119047619, 0.6586681547619048], [0.4720982142857143, 0.5892857142857143, 0.6389508928571429, 0.6065848214285714, 0.67578125, 0.6612723214285714, 0.6177455357142857, 0.654389880952381, 0.650483630952381, 0.6391369047619048]]


In [12]:
print(overall_list_dev_precision)

[[0.442030839735173, 0.5046146666094974, 0.5232619713325154, 0.5104209406664764, 0.5488831580126223, 0.5489835520531977, 0.5509874185266365, 0.5677201639688508, 0.5617961267595001, 0.5663702100886974], [0.42639442877393396, 0.5011038586471886, 0.4709326627841666, 0.4987377141550412, 0.5663058792289154, 0.5632999520428342, 0.5550538800433068, 0.5500791448879269, 0.5573855856445141, 0.5628942300987995], [0.31057236049247317, 0.4801525950062273, 0.4988348436472572, 0.5253474855996513, 0.5396285011673666, 0.5562299094547339, 0.5558410465001852, 0.5729029923449566, 0.5647138147945656, 0.5493747512116733], [0.4478261080686722, 0.48919493361692606, 0.5197687932253321, 0.5436942212015465, 0.5628443846100579, 0.5792726544289044, 0.5937577319757494, 0.5987977947512583, 0.5784954914021493, 0.5809145150400058], [0.4374434391803858, 0.475445317365997, 0.5033520829392056, 0.5428483093869437, 0.6018983138855774, 0.5786879798155921, 0.5547009404723426, 0.5782985327151348, 0.5797638076938235, 0.5812848

In [13]:
print(overall_list_dev_recall)

[[0.451985438563384, 0.5055002259499372, 0.5371169666262408, 0.5364604264406967, 0.5599809727289272, 0.5631556909124219, 0.5582297245284447, 0.577560375111583, 0.5442599351228345, 0.5406427271750623], [0.40557624725298036, 0.5200598547295251, 0.5131585212948203, 0.5083875637959147, 0.5604986848323245, 0.5585703607599337, 0.5685833384620563, 0.5168016578717735, 0.5650435002396249, 0.5852699663052868], [0.322754453551909, 0.5094413000984346, 0.5029123544085465, 0.5210633280195283, 0.571027443362175, 0.5570852788805988, 0.5515902412829932, 0.5720596365002876, 0.5558932588186789, 0.5426841345512564], [0.4586726502955495, 0.5123842037696874, 0.548661637950635, 0.5316042608039718, 0.5615388095797759, 0.5673564080505837, 0.5667669703370859, 0.557032415238823, 0.5739211067815668, 0.5712829337737425], [0.37384405167952967, 0.505838180551941, 0.521993056255136, 0.5144091372951667, 0.5816390531338287, 0.5805384205692767, 0.5585078935420321, 0.5648700101858618, 0.5816818866241134, 0.58200713924847

In [14]:
print(overall_list_dev_f1score)

[[0.4269054104971343, 0.4849079689047987, 0.5131872984464014, 0.5025273025962698, 0.5342392955402305, 0.537457250152518, 0.5315243291663415, 0.5501200143496333, 0.5360687919041977, 0.5318716775134587], [0.3747147327796451, 0.4950079186851356, 0.47365121160456614, 0.46995429321476256, 0.5426296738568918, 0.5407339846067538, 0.5327709871407993, 0.5066764210022235, 0.5397020415498816, 0.5525772993219212], [0.2919721934799813, 0.4818844468041257, 0.4761812830820948, 0.4999058474667876, 0.5327105114497164, 0.534672628164551, 0.5330834361351549, 0.5484651800948048, 0.5418468206733614, 0.5231602610695005], [0.43923865621966024, 0.48452489880899063, 0.5168683803489783, 0.5152169434463821, 0.5379836715171818, 0.5515601470365655, 0.5547870104138026, 0.5500670964956272, 0.5538546753238865, 0.5548791058831062], [0.35777600379168467, 0.4696399021425645, 0.4944058376960355, 0.4875014768024037, 0.5735926593972456, 0.5525664570447456, 0.5358198224441646, 0.5500869897500145, 0.5599047526104928, 0.56078

In [15]:
#The best model is 3